In [2]:
import os
import calendar
import datetime
import polars as pl
from os import listdir
from os.path import isfile, join
from utils.functions import tramit_file_reader
from utils.dictionaries import types_parque_post
from utils.dictionaries import common_cols_tramites, common_cols_mapping, common_cols_parque_exacto

Juntar todos los tramites

In [ ]:
path = os.path.join("..","Data", "DGT")
matriculaciones_path = os.path.join(path,'matr')
files_matr = [
    f for f in os.listdir(matriculaciones_path)
    if os.path.isfile(os.path.join(matriculaciones_path, f)) and f.lower().endswith(".txt")
]

first_matr_file_path = os.path.join(matriculaciones_path,files_matr[0])
file_matr = tramit_file_reader(first_matr_file_path).select(common_cols_tramites).rename(common_cols_mapping)
files_len_matr = len(files_matr)

for i in range(1,files_len_matr):
    file_path = os.path.join(matriculaciones_path,files_matr[i])
    new_file = tramit_file_reader(file_path).select(common_cols_tramites).rename(common_cols_mapping)
    file_matr = pl.concat([file_matr,new_file])

matriculaciones_file_path = os.path.join(matriculaciones_path,'matriculaciones.csv')
file_matr.write_csv(matriculaciones_file_path,separator='|')

bajas_path = os.path.join(path,'bajas')
bajas_files = [
    f for f in os.listdir(bajas_path)
    if os.path.isfile(os.path.join(bajas_path, f)) and f.lower().endswith(".txt")
]

first_file_path = os.path.join(bajas_path,bajas_files[0])
file_bajas = tramit_file_reader(first_file_path).select(common_cols_tramites).rename(common_cols_mapping)
files_len = len(bajas_files)

for i in range(1,files_len):
    file_path = os.path.join(bajas_path,bajas_files[i])
    new_file = tramit_file_reader(file_path).select(common_cols_tramites).rename(common_cols_mapping)
    file_bajas = pl.concat([file_bajas,new_file])

baja_file_path = os.path.join(bajas_path,'bajas.csv')
file_bajas.write_csv(baja_file_path,separator='|')

Juntar todas las bajas

In [3]:
def dates_range(start, end,type):
    dates = []
    current = datetime.date(start.year, start.month, 1)
    while current <= end:
        if type == 'mat':
            dates.append(f"export_mensual_mat_{current.year}{current.month:02d}.txt")
        elif type == 'bajas':
            dates.append(f"export_mensual_bajas_{current.year}{current.month:02d}.txt")
        if current.month == 12:
            current = datetime.date(current.year + 1, 1, 1)
        else:
            current = datetime.date(current.year, current.month + 1, 1)
    return dates

def simplify_euro_emissions(df):
    df = df.with_columns([
        pl.when(pl.col("FECHA_PRIM_MATR") >= pl.lit(datetime.date(2015, 9, 1))).then(pl.lit("EURO_6"))
        .when(pl.col("FECHA_PRIM_MATR") >= pl.lit(datetime.date(2011, 1, 1))).then(pl.lit("EURO_5"))
        .when(pl.col("FECHA_PRIM_MATR") >= pl.lit(datetime.date(2006, 1, 1))).then(pl.lit("EURO_4"))
        .when(pl.col("FECHA_PRIM_MATR") >= pl.lit(datetime.date(2001, 1, 1))).then(pl.lit("EURO_3"))
        .when(pl.col("FECHA_PRIM_MATR") >= pl.lit(datetime.date(1997, 1, 1))).then(pl.lit("EURO_2"))
        .when(pl.col("FECHA_PRIM_MATR") >= pl.lit(datetime.date(1992, 12, 31))).then(pl.lit("EURO_1"))
        .when(pl.col("FECHA_PRIM_MATR") < pl.lit(datetime.date(1992, 12, 31))).then(pl.lit("Previous"))
        .otherwise(pl.lit("Unkwown"))
        .alias("Simplified_EURO")
        ])

    df = df.with_columns(
        pl.when((pl.col("PROPULSION") == "ELEC") | (pl.col("PROPULSION") == "H"))
        .then(pl.lit("EURO_CLEAN"))
        .otherwise(pl.col("Simplified_EURO"))
        .alias("Simplified_EURO")
        )

    return df

def get_cars(fecha,park,
             mat_path = os.path.join("..","Data", "DGT",'matr'),
             bajas_path = os.path.join("..","Data", "DGT",'bajas')):
    '''
    fecha : date in string with format %d%m%Y
    ''' 
    fecha_foto = park.select(pl.max("FECHA_MATR")).item()
    fecha = datetime.datetime.strptime(fecha, "%d%m%Y").date()

    if fecha >= fecha_foto:
        # Todos los coches en parque_exacto +
        # todas las matr entre (parque_exacto, fecha) -
        # todas las bajas entre (parque_exacto, fecha) con fecha_prim_matr anterior a fecha
        park_distribution = park['Simplified_EURO'].value_counts().sort('Simplified_EURO')#.rename({"count": "count1"})
        mat_files = dates_range(fecha_foto,fecha,'mat')
        bajas_files = dates_range(fecha_foto,fecha,'bajas')
        for file in mat_files:
            file_path = os.path.join(mat_path,file)
            file = tramit_file_reader(file_path).select(common_cols_tramites).rename(common_cols_mapping)
            file = file.with_columns([
                pl.when(pl.col("FECHA_PRIM_MATR").is_null()).then(pl.col("FECHA_MATR"))
                .otherwise(pl.col("FECHA_PRIM_MATR"))
                .alias("FECHA_PRIM_MATR")
                ])
            file = simplify_euro_emissions(file)
            file = file['Simplified_EURO'].value_counts().sort('Simplified_EURO')#.rename({"count": "count2"})
            park_distribution = park_distribution.join(file, on="Simplified_EURO", how="full")
            park_distribution = park_distribution.fill_null(0).with_columns([
                (pl.col("count") + pl.col("count_right")).alias("count")
                ])
            park_distribution = park_distribution.drop(["Simplified_EURO_right","count_right"])

        for file in bajas_files:
            file_path = os.path.join(bajas_path,file)
            file = tramit_file_reader(file_path).select(common_cols_tramites).rename(common_cols_mapping)
            file.filter(pl.col("FECHA_PRIM_MATR") <= fecha)
            file = file.with_columns([
                pl.when(pl.col("FECHA_PRIM_MATR").is_null()).then(pl.col("FECHA_MATR"))
                .otherwise(pl.col("FECHA_PRIM_MATR"))
                .alias("FECHA_PRIM_MATR")
                ])
            file = simplify_euro_emissions(file)
            file = file['Simplified_EURO'].value_counts().sort('Simplified_EURO')#.rename({"count": "count2"})
            park_distribution = park_distribution.join(file, on="Simplified_EURO", how="full")
            park_distribution = park_distribution.fill_null(0).with_columns([
                (pl.col("count") - pl.col("count_right")).alias("count")
                ])
            park_distribution = park_distribution.drop(["Simplified_EURO_right","count_right"])            

        return park_distribution

    elif fecha_foto >= fecha:
        # Todos los coches en parque_exacto con fecha_prim_matr < fecha +
        # Todas las bajas entre fehca y fecha_foto con fecha_prim_matr < fecha
        park = park.filter(pl.col("FECHA_PRIM_MATR") <= fecha)
        park_distribution = park['Simplified_EURO'].value_counts().sort('Simplified_EURO')#.rename({"count": "count1"})
        bajas_files = dates_range(fecha,fecha_foto,'bajas')

        for file in bajas_files:
            file_path = os.path.join(bajas_path,file)
            file = tramit_file_reader(file_path).select(common_cols_tramites).rename(common_cols_mapping)
            file.filter(pl.col("FECHA_PRIM_MATR") <= fecha)
            file = file.with_columns([
                pl.when(pl.col("FECHA_PRIM_MATR").is_null()).then(pl.col("FECHA_MATR"))
                .otherwise(pl.col("FECHA_PRIM_MATR"))
                .alias("FECHA_PRIM_MATR")
                ])
            file = simplify_euro_emissions(file)
            file = file['Simplified_EURO'].value_counts().sort('Simplified_EURO')#.rename({"count": "count2"})
            park_distribution = park_distribution.join(file, on="Simplified_EURO", how="full")
            park_distribution = park_distribution.fill_null(0).with_columns([
                (pl.col("count") + pl.col("count_right")).alias("count")
                ])
            park_distribution = park_distribution.drop(["Simplified_EURO_right","count_right"])

        return park_distribution

In [11]:
clean_park = os.path.join("..","Data", "DGT",'Parque_exacto','clean_park.csv')
park= pl.scan_csv(clean_park,separator='|', schema=types_parque_post).select(common_cols_parque_exacto)
park = park.collect()
park = simplify_euro_emissions(park)

In [ ]:
example = get_cars('01022024',park)

In [23]:
dates = ['01062023','01072023','01082023','01092023','01102023','01112023','01122023','01012024','01022024','01032024','01042024','01052024','01062024','01012024']
vehicles = []
for date in dates:
    vehicles.append(get_cars(date,park)["count"].sum())
print(vehicles)

[38450837, 38444488, 38358935, 38331545, 38259027, 38234901, 38194674, 37595154, 36903019, 36924798, 36933420, 36946191, 36984382, 37595154]


In [ ]:
clean_park = os.path.join("..","Data", "DGT",'Parque_exacto','clean_park.csv')
park = pl.scan_csv(clean_park,separator='|', schema=types_parque_post).select(common_cols_parque_exacto)
park = park.collect()
park = simplify_euro_emissions(park)
resum_file = file['Simplified_EURO'].value_counts()
resum_park = park['Simplified_EURO'].value_counts()

## Dummy scenario

In [4]:
mat_dummy1 = dates_range(
    datetime.date(2023,11,1),
    datetime.date(2024,1,1),
    'mat')

baja_dummy1 = dates_range(
    datetime.date(2023,11,1),
    datetime.date(2024,1,1),
    'bajas')

In [6]:
baja_dummy1

['export_mensual_bajas_202311.txt',
 'export_mensual_bajas_202312.txt',
 'export_mensual_bajas_202401.txt']

In [ ]:
mat_dummy1 = dates_range(
    datetime.date(2023,11,1),
    datetime.date(2024,1,1),
    'mat')

baja_dummy1 = dates_range(
    datetime.date(2023,11,1),
    datetime.date(2024,1,1),
    'bajas')

mat_dummy2 = dates_range(
    datetime.date(2022,1,1),
    datetime.date(2022,3,1),
    'mat')

baja_dummy2 = dates_range(
    datetime.date(2022,1,1),
    datetime.date(2022,3,1),
    'bajas')

mat_dummy = mat_dummy1 #+ mat_dummy2
bajas_dummy = baja_dummy1 #+ baja_dummy2

mat_path = os.path.join("..","Data", "DGT",'matr')
bajas_path = os.path.join("..","Data", "DGT",'bajas')
dummy_path_mat = os.path.join("..","Data", "DGT",'dummy','mat')
dummy_path_bajas = os.path.join("..","Data", "DGT",'dummy','bajas')

for file in mat_dummy:
    input_path = os.path.join(mat_path,file)
    output_path = os.path.join(dummy_path_mat,file)
    with open(input_path, 'r', encoding='ISO-8859-1') as infile:
        lines = [next(infile) for _ in range(5) if infile]
            
    with open(output_path, 'w', encoding='ISO-8859-1') as outfile:
        outfile.writelines(lines)

for file in bajas_dummy:
    input_path = os.path.join(bajas_path,file)
    output_path = os.path.join(dummy_path_bajas,file)
    with open(input_path, 'r', encoding='ISO-8859-1') as infile:
        lines = [next(infile) for _ in range(5) if infile]
            
    with open(output_path, 'w', encoding='ISO-8859-1') as outfile:
        outfile.writelines(lines)

clean_park = os.path.join("..","Data", "DGT",'Parque_exacto','clean_park.csv')
output_path = os.path.join("..","Data", "DGT","dummy","Parque_exacto","clean_park.csv")
with open(clean_park, 'r', encoding='utf-8') as infile:
    lines = [next(infile) for _ in range(5) if infile]

with open(output_path, 'w', encoding='utf-8') as outfile:
    outfile.writelines(lines)

In [34]:
mat_dummy2

['export_mensual_mat_202201.txt',
 'export_mensual_mat_202202.txt',
 'export_mensual_mat_202203.txt']

In [27]:
mat11 = os.path.join(dummy_path_mat,mat_dummy1[0])
tramit_file_reader(mat11).select(common_cols_tramites).rename(common_cols_mapping)

FECHA_MATR,FECHA_PRIM_MATR,CLASE_MATR,MARCA,MODELO,PROCEDENCIA,TIPO,TIPO_DGT,PROPULSION,CILINDRADA,POTENCIA,KW,TARA,PESO_MAX,MOM,MMTA,PLAZAS,PLAZAS_MAX,PLAZAS_PIE,NUM_TITULARES,PROVINCIA,MUNICIPIO,PROVINCIA_MATR,NUEVO_USADO,TIPO_TITULAR,SERVICIO,EMISIONES_CO2,RENTING,VARIANTE,VERSION,FABRICANTE,CARROCERIA,DISTANCIA_EJES,EJE_ANTERIOR,EJE_POSTERIOR,EMISIONES_EURO,CONSUMO,AUTONOMIA,ALIMENTACION,CATELECT
date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2024-01-02,null,"""0""","""TOYOTA""","""TOYOTA C-HR""","""3""","""40""","""""","""0""","""1987""","""13.27""","""112.00""","""0""","""1955""","""1550""","""1955""","""5""","""5""","""0""","""01""","""M""","""28065""","""M""","""N""","""D""","""B00""","""109""","""N""","""MAXH20(H)""","""MAXH20L-AHXGBW(1A)""","""TOYOTA MOTOR CORPORATION""","""AB""","""2640""","""1570""","""1585""","""=6EA""","""0""","""000000""","""M""","""HEV"""
2024-01-02,2010-06-18,"""0""","""FIAT""","""FI""","""3""","""7A""","""""","""1""","""2287""","""14.43""","""95.50""","""3075""","""3500""","""3075""","""3500""","""4""","""4""","""0""","""01""","""CO""","""14038""","""T""","""U""","""D""","""B00""","""""","""N""","""FIF1""","""FEM4HI0G04BDI694""","""FIAT GROUP AUTOMOBILES S.P.A.""","""SA""","""4035""","""1810""","""1980""","""EURO 4""","""0""","""000000""","""M""",""""""
2024-01-02,null,"""0""","""RODRIGUEZ LOPEZ AUTO""","""RLA TRANSIT L3H3""","""0""","""22""","""""","""1""","""1995""","""13.29""","""125.00""","""0""","""3500""","""""","""""","""5""","""5""","""""","""01""","""SE""","""41091""","""SE""","""N""","""X""","""A07""","""234""","""N""","""ABLFB1ADX""","""WWTASAABA3AB32A01B""","""RODRIGUEZ LOPEZ AUTO""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
2024-01-02,null,"""0""","""JEEP""","""RENEGADE""","""3""","""40""","""""","""0""","""1469""","""11.07""","""96.00""","""0""","""1990""","""1495""","""1990""","""5""","""5""","""0""","""01""","""M""","""28046""","""M""","""N""","""X""","""A01""","""129""","""N""","""AXW14""","""E22DP""","""FCA US LLC""","""AF""","""2570""","""1551""","""1553""","""EURO 6AP""","""0""","""000000""","""M""","""HEV"""


In [28]:
mat11 = os.path.join(dummy_path_mat,mat_dummy1[1])
tramit_file_reader(mat11).select(common_cols_tramites).rename(common_cols_mapping)

FECHA_MATR,FECHA_PRIM_MATR,CLASE_MATR,MARCA,MODELO,PROCEDENCIA,TIPO,TIPO_DGT,PROPULSION,CILINDRADA,POTENCIA,KW,TARA,PESO_MAX,MOM,MMTA,PLAZAS,PLAZAS_MAX,PLAZAS_PIE,NUM_TITULARES,PROVINCIA,MUNICIPIO,PROVINCIA_MATR,NUEVO_USADO,TIPO_TITULAR,SERVICIO,EMISIONES_CO2,RENTING,VARIANTE,VERSION,FABRICANTE,CARROCERIA,DISTANCIA_EJES,EJE_ANTERIOR,EJE_POSTERIOR,EMISIONES_EURO,CONSUMO,AUTONOMIA,ALIMENTACION,CATELECT
date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2024-02-01,null,"""0""","""TOYOTA""","""PROACE CITY""","""3""","""20""","""""","""1""","""1499""","""11.20""","""75.00""","""0""","""2020""","""1446""","""2020""","""2""","""2""","""0""","""01""","""M""","""28178""","""SE""","""N""","""X""","""B00""","""144""","""N""","""F""","""YHT2-G2F021(1T)""","""TOYOTA MOTOR EUROPE NV/SA""","""BB""","""2785""","""1563""","""1577""","""EURO 6AQ""","""0""","""000000""","""M""",""""""
2024-02-01,null,"""0""","""HARLEY-DAVIDSON""","""SPORTSTER S""","""3""","""50""","""""","""0""","""1252""","""10.48""","""""","""0""","""418""","""""","""""","""1""","""""","""""","""01""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
2024-02-01,null,"""0""","""TOYOTA""","""TOYOTA COROLLA""","""3""","""40""","""""","""0""","""1798""","""12.49""","""72.00""","""0""","""1885""","""1475""","""1885""","""5""","""5""","""0""","""01""","""M""","""28006""","""M""","""N""","""X""","""B00""","""101""","""S""","""ZWE219(W)""","""ZWE219L-DWXNBW(6H)""","""TOYOTA MOTOR EUROPE NV/SA""","""AC""","""2700""","""1530""","""1530""","""EURO 6AP""","""0""","""000000""","""M""","""HEV"""
2024-02-01,null,"""0""","""TESLA""","""MODEL Y""","""3""","""40""","""""","""2""","""0""","""36.87""","""378.00""","""0""","""2518""","""""","""""","""5""","""5""","""""","""01""","""MU""","""30030""","""M""","""N""","""D""","""B00""","""""","""N""","""Y5LD""","""BVB3S5T3""","""TESLA""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [31]:
mat11 = os.path.join(dummy_path_mat,mat_dummy1[2])
tramit_file_reader(mat11).select(common_cols_tramites).rename(common_cols_mapping)

FECHA_MATR,FECHA_PRIM_MATR,CLASE_MATR,MARCA,MODELO,PROCEDENCIA,TIPO,TIPO_DGT,PROPULSION,CILINDRADA,POTENCIA,KW,TARA,PESO_MAX,MOM,MMTA,PLAZAS,PLAZAS_MAX,PLAZAS_PIE,NUM_TITULARES,PROVINCIA,MUNICIPIO,PROVINCIA_MATR,NUEVO_USADO,TIPO_TITULAR,SERVICIO,EMISIONES_CO2,RENTING,VARIANTE,VERSION,FABRICANTE,CARROCERIA,DISTANCIA_EJES,EJE_ANTERIOR,EJE_POSTERIOR,EMISIONES_EURO,CONSUMO,AUTONOMIA,ALIMENTACION,CATELECT
date,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2024-03-01,null,"""0""","""TOYOTA""","""TOYOTA YARIS CROSS""","""3""","""40""","""""","""0""","""1490""","""9.95""","""68.00""","""0""","""1690""","""1250""","""1690""","""5""","""5""","""0""","""01""","""M""","""28169""","""M""","""N""","""X""","""A01""","""101""","""S""","""MXPJ10(H)""","""MXPJ10L-BHXNBW(DD)""","""TOYOTA MOTOR CORPORATION""","""AB""","""2560""","""1520""","""1520""","""EURO 6AP""","""0""","""000000""","""M""","""HEV"""
2024-03-01,null,"""6""","""SCOOBIC""","""SCOOBIC LIGHT""","""3""","""91""","""""","""2""","""0""","""0.71""","""3.70""","""265""","""685""","""265""","""685""","""1""","""1""","""0""","""01""","""SE""","""41038""","""SE""","""N""","""X""","""A01""","""""","""N""","""1""","""9""","""GREEN MESSENGER ELECTRIC VEHIC…","""""","""2240""","""0""","""820""","""EURO 5""","""0""","""000000""","""""",""""""
2024-03-01,null,"""0""","""MAN""","""TGX""","""3""","""81""","""""","""1""","""12419""","""46.80""","""382.00""","""0""","""18000""","""8003""","""18000""","""2""","""2""","""0""","""01""","""M""","""28006""","""M""","""N""","""X""","""A01""","""""","""S""","""06K04ZBBAAA1SAA""","""G0ACF""","""MAN TRUCK & BUS SE""","""BC""","""3600""","""0""","""0""","""=VIE""","""0""","""000000""","""""",""""""
2024-03-01,null,"""0""","""HYUNDAI""","""TUCSON""","""3""","""40""","""""","""1""","""1598""","""11.64""","""100.00""","""0""","""2140""","""1606""","""2140""","""5""","""5""","""0""","""01""","""M""","""28169""","""B""","""N""","""X""","""B00""","""134""","""S""","""F5D51""","""D72A14""","""HYUNDAI MOTOR MANUFACTURING CZ…","""AF""","""2680""","""1630""","""1637""","""EURO 6AP""","""0""","""000000""","""M""","""HEV"""
